# External Mail Forwarding Rule (BEC)

**Mailbox rules forwarding to external recipients—common in business email compromise.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **rule creator**, **creation IP/device**, **recent sign-ins**, **OAuth apps**, **delegates**.


## Investigation Queries

Find inbox rule creation/modification and identify external forward target.


In [ ]:
// KQL
OfficeActivity
| where TimeGenerated > ago(90d)
| where UserId == '<USER>'
| where Operation in ('New-InboxRule','Set-InboxRule')
| project TimeGenerated, UserId, Operation, ClientIP, Parameters


In [ ]:
# Splunk SPL
index=o365 user=<USER> Operation IN ("New-InboxRule","Set-InboxRule")
| table _time user Operation ClientIP Parameters


In [ ]:
# Elastic (KQL/EQL)
event.category:email AND event.action:(New-InboxRule OR Set-InboxRule) AND user.name:"<USER>"


## Containment & Response

Remove the rule, revoke tokens, reset password, and check for additional persistence (OAuth grants).


In [ ]:
# PowerShell
## Remove rule (Exchange Online PowerShell; example)
# Get-InboxRule -Mailbox <USER>
# Remove-InboxRule -Mailbox <USER> -Identity "<RULE_NAME>" -Confirm:$false


In [ ]:
# PowerShell
## Revoke Azure AD tokens
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>
